In [ ]:
# Load keypoints as csv
import pandas as pd




In [ ]:
"""
Schema of the data:
- 6 columns for the angles sequences
- exercise name
"""
df = pd.read_csv("full_video_dataset.csv")

df

ParserError: ignored

In [ ]:
# Encode the labels
label_to_idx = {}

idx_to_label = {}

for label in df.exercise_label.unique():
  label_to_idx[label] = len(label_to_idx)
  idx_to_label[label_to_idx[label]] = label


In [ ]:
NUM_CLASSES = len(idx_to_label)

print("Number of classes", NUM_CLASSES)

Number of classes 6


In [ ]:
print(idx_to_label)

{0: 'Deadlift', 1: 'Biceps curl', 2: 'Push-up', 3: 'Tricep Pushdown', 4: 'Lat pulldown', 5: 'Incline bench press'}


In [ ]:
import torch
from torch import nn

In [ ]:
def frame_to_input(row):
  # HERE
  keypoints = row["right_elbow_angle":"left_knee_angle"]

  # result = 
  results = []
  for keypoint in keypoints:
    results.append(list(map(float, keypoints[0].replace("(", "").replace(")", "").split(","))))
  # 8 angle values
  # print(len(results))

  label = label_to_idx[row["exercise_label"]]
  return [results, label]

lengthes = set()

dataset = df.apply(frame_to_input, axis=1)

reformatted_dataset = []

for data, label in dataset:
  multivariate_timeseries = list(zip(*data))

  lengthes.add(len(multivariate_timeseries))

  for i in range(len(multivariate_timeseries)):
    multivariate_timeseries[i] = list(multivariate_timeseries[i])

  reformatted_dataset.append([multivariate_timeseries, label])

print(len(reformatted_dataset))
print(len(reformatted_dataset[0]))
print(len(reformatted_dataset[0][0]))
print(len(reformatted_dataset[0][0][0]))

reformatted_data, labels = list(zip(*reformatted_dataset))

print(len(reformatted_data))
print(len(labels))


216
2
50
8
216
216


In [ ]:
print(len(dataset))

216


In [ ]:
from torch.utils.data import DataLoader , Dataset


class CustomDataset(Dataset):
  def __init__(self, dataset, labels):
    self.dataset = dataset
    self.labels = labels

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return torch.tensor(self.dataset[idx]), torch.tensor(self.labels[idx])

In [ ]:
custom_dataset = CustomDataset(reformatted_data, labels)

data_loader = DataLoader(custom_dataset, batch_size=1, shuffle=True)

print(len(custom_dataset))

216


In [ ]:
# NUM_CLASSES = 8

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable 

class VideoExerciseClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    # self.activation = nn.Tanh()

    self.num_layers = 2

    self.hidden_size = 8
    self.input_size = 50

    self.relu = nn.ReLU()

    self.lstm = nn.LSTM(8, 8, num_layers=2, batch_first=True)
    self.dense = nn.Linear(self.hidden_size, NUM_CLASSES)
    # self.dense_2 = nn.Linear(12, NUM_CLASSES)

    

  # def forward(self, input):
  #   out = self.dense(input)
  #   out = self.activation(out)
  #   out = self.dense_2(out)

  #   return out

  def forward(self,x):
    h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)) #hidden state
    c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)) #internal state
    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
    # print(hn.size())
    # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    out = self.relu(hn[-1])
    out = self.dense(out) #first Dense
    # out = self.relu(out) #relu
    # out = self.fc(out) #Final Output
    # print(out.size())
    return out

In [ ]:
# training loop
model = VideoExerciseClassifier().to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.005)

loss = nn.CrossEntropyLoss()

def train():
  model.train()

  losses = []
  for batch, labels in data_loader:
    output = model(batch.to(device))

    l = loss(output, labels.to(device))

    losses.append(l.item())

    l.backward()
    optimizer.step()
    model.zero_grad()

  print("Mean Loss", sum(losses)/len(losses))

In [ ]:
def frame_accuracy(logits, labels):
  probs = nn.functional.softmax(logits, dim=1)
  preds = torch.argmax(probs, dim=1)

  # print(preds)

  return sum([1 if preds[i].item() == label.item() else 0 for i, label in enumerate(labels)])/labels.size()[0]


def evaluate():
  model.eval()
  accuracies = []
  for batch, labels in data_loader:
    output = model(batch.to(device))

    accuracies.append(frame_accuracy(output, labels.to(device)))

  mean = sum(accuracies)/len(accuracies)

  print("Mean accuracy", mean)


In [ ]:
evaluate()
for i in range(200):
  print("Epoch", i+1)
  train()
  evaluate()

Mean accuracy 0.041666666666666664
Epoch 1
Mean Loss 1.709089113054452
Mean accuracy 0.6898148148148148
Epoch 2
Mean Loss 1.364462550315592
Mean accuracy 0.6898148148148148
Epoch 3
Mean Loss 1.1656380008768152
Mean accuracy 0.6898148148148148
Epoch 4
Mean Loss 1.0613048054553844
Mean accuracy 0.6898148148148148
Epoch 5
Mean Loss 1.0049903784637098
Mean accuracy 0.6898148148148148
Epoch 6
Mean Loss 0.9682757210124422
Mean accuracy 0.6898148148148148
Epoch 7
Mean Loss 0.936291566601506
Mean accuracy 0.6898148148148148
Epoch 8
Mean Loss 0.9066841362251176
Mean accuracy 0.6898148148148148
Epoch 9
Mean Loss 0.8820123014350733
Mean accuracy 0.6898148148148148
Epoch 10
Mean Loss 0.8592099592917495
Mean accuracy 0.6898148148148148
Epoch 11
Mean Loss 0.8368066741084611
Mean accuracy 0.6898148148148148
Epoch 12
Mean Loss 0.8134984848675905
Mean accuracy 0.6898148148148148
Epoch 13
Mean Loss 0.7904381787887326
Mean accuracy 0.6898148148148148
Epoch 14
Mean Loss 0.7718590752532085
Mean accuracy 0.

In [ ]:
torch.save(model.state_dict(), "exercise_classifier.pt")